In [2]:
%%capture
%pip install -q transformers datasets nltk spacy seaborn matplotlib Wikipedia-API

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
def split_list(input_list, chunk_size):
    result = []
    for i in range(0, len(input_list), chunk_size):
        result.append(input_list[i:i + chunk_size])
    return result

input_list = list(range(23))  # A list with 23 elements
chunk_size = 10  # Set the chunk size
output_lists = split_list(input_list, chunk_size)

for i, lst in enumerate(output_lists):
    print(f"Split list {i}: {lst}")

Split list 0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Split list 1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Split list 2: [20, 21, 22]


In [5]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large').to(device)

In [7]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('MyProjectName (merlin@example.com)', 'ru')
page = wiki_wiki.page('Россия')

text = page.text

In [8]:
from nltk import word_tokenize

In [50]:
sentences = [token for token in text.split('\n') if len(word_tokenize(token)) > 7]

In [51]:
tokens = []

database = {}

formatted_queries = [f'query: {sentence}' for sentence in sentences]

In [52]:
tokenized_sentences = tokenizer(
    formatted_queries, max_length=512,
    padding=True, truncation=True,
    return_tensors='pt'
)

In [53]:
tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(device)
tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(device)

In [54]:
from torch import Tensor

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [55]:
def batch_split(input_list, batch_size=8):
    result = []
    for i in range(0, len(input_list), (batch_size)):
        result.append(input_list[i:i + batch_size])
    return result

In [56]:
from tqdm.auto import tqdm

model.eval()
outputs = []

with torch.no_grad():
    batches = batch_split(range(len(formatted_queries)), batch_size=8)
    for i in tqdm(batches):
        batch = {'input_ids': tokenized_sentences['input_ids'][i[0]:i[-1] + 1], 'attention_mask': tokenized_sentences['attention_mask'][i[0]:i[-1] + 1]}
        out = model(**batch)
        embeddings = average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'][i[0]:i[-1] + 1])
        outputs.append(embeddings.squeeze(0))

  0%|          | 0/55 [00:00<?, ?it/s]

In [57]:
outputs = torch.cat(outputs, dim=0).detach().cpu()

In [58]:
import torch.nn.functional as F

embeddings = F.normalize(outputs, p=2, dim=1)

In [59]:
assert embeddings.shape[0] == len(sentences)

In [60]:
for i, sentence in enumerate(sentences):
    database[sentence] = embeddings[i]

In [61]:
import pickle

with open('embeddings_db.pkl', 'wb') as f:
    pickle.dump(database, f)

In [62]:
from typing import List


def rank(sentence: str, k: int=5) -> List[str]:
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    sentence = f'query: {sentence}'
    tokenized_sentences = tokenizer(
        [sentence], max_length=512,
        padding=True, truncation=True,
        return_tensors='pt'
    )
    tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(device)
    tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(device)
    out = model(**tokenized_sentences)
    embeddings = average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'])
    embeddings = embeddings.detach().cpu()
    search_metrics = {}

    for sentence in database.keys():
        metric = F.cosine_similarity(embeddings, database[sentence])
        search_metrics[sentence] = metric.item()

    sorted_scores = sorted(search_metrics.items(), key=lambda x: x[1], reverse=True)
    return [(sentence, score) for sentence, score in sorted_scores[:k]]

In [71]:
rank('Выборы 2018 года', 5)

[(' — от 2 000 000 до 6 000 000 чел.', 0.8186419606208801),
 ('Президент избирается сроком на шесть лет тайным голосованием на всеобщих прямых выборах, один и тот же человек не может занимать президентский пост более двух сроков. Конституционные нормы, предусматривающие шестилетний срок полномочий президента, введены в 2008 году, ранее глава государства избирался раз в четыре года.',
  0.8157840371131897),
 (' — от 1 000 000 до 2 000 000 чел.', 0.8157546520233154),
 ('Федеральная служба войск национальной гвардии Российской Федерации;',
  0.8150832056999207),
 (' — от 600 000 до 1 000 000 чел.', 0.8130089640617371)]

In [72]:
rank('Августовский путч')

[('19—21 августа 1991 года в Москве произошёл «Августовский путч», вызвавший противостояние властей СССР и РСФСР, что привело к массовым демонстрациям у Белого дома в поддержку президента РСФСР Б. Н. Ельцина. Попытку госпереворота организовали деятели партии, КГБ и военные, пытавшиеся предотвратить распад СССР. Непродуманный и плохо выполненный заговор положил конец КПСС и только ускорил развал государства. 8 декабря 1991 года были подписаны Беловежские соглашения о прекращении существования СССР и создании СНГ.',
  0.8214746117591858),
 ('8 августа 2008 года началась война в Грузии, после которого Россия официально признала Абхазию и Южную Осетию в качестве независимых государств.',
  0.7753043174743652),
 ('Столица — Петроград, с 12 марта 1918 года — Москва.', 0.7751259803771973),
 ('1 мая — Праздник весны и труда;', 0.7730187177658081),
 ('В первой половине 1990-х годов большое количество предприятий было приватизировано путём ваучерной приватизации, а также через залоговые аукционы

In [74]:
rank('отставка Ельцина')

[('31 декабря 1999 года Борис Ельцин объявил об отставке с поста президента, назначив исполняющим обязанности президента председателя правительства России В. В. Путина.',
  0.8573992252349854),
 ('Президент может быть отрешён от должности Советом Федерации при условии выдвижения Государственной думой обвинения в государственной измене или совершения иного тяжкого преступления и наличия положительных заключений Верховного и Конституционного судов.',
  0.813132643699646),
 ('Главой государства является президент Российской Федерации, в настоящее время — Владимир Владимирович Путин. В исполнении обязанностей главы государства президенту оказывает содействие председатель Правительства Российской Федерации, в настоящее время им является Михаил Владимирович Мишустин. Председатель Правительства России занимает пост президента в случае смерти или отставки последнего.',
  0.8018243908882141),
 ('Президент обладает рядом важнейших полномочий: руководит внешней политикой страны, является Верховны